# Supervised Learning Models for Anomaly-Based Intrusion Detection

This Jupyter Notebook focuses on training and evaluating supervised machine learning models for anomaly-based intrusion detection. It builds upon the preprocessed CICIDS2017 dataset, prepared in the **data preprocessing notebook

The broader goal is to develop a Network Intrusion Detection System (NIDS) prototype capable of identifying a range of network attacks, such as DoS, PortScan, and Brute Force, while balancing detection accuracy with computational efficiency. This is particularly critical in resource-constrained environments, focus of the project.

**Models Being Evaluated:**

* **Supervised Learning:**
    * Random Forest
    * XGBoost
    * _k_-NN

**Evaluation Strategy:**

The models are assessed using k-fold cross-validation on the training data, with a separate hold-out test set reserved for final evaluation. Key performance metrics include accuracy, precision, recall, F1-score, and resource usage (CPU time and memory consumption). These metrics provide critical insights into algorithm efficiency and effectiveness, essential for real-time deployment in resource-constrained networks.

This notebook presents the training process, hyperparameter tuning, and comparative analysis of the selected supervised learning models.

In [ ]:
from google.colab import drive

In [ ]:
 drive.mount('/content/drive')

In [ ]:
# Installing extra components
!pip install memory_profiler
!pip install psutil

In [ ]:
# Importing the relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

import time
import psutil
import threading
from memory_profiler import memory_usage

import joblib

# Helper Functions

In [ ]:
# Apply RF with resource measurements
def apply_rf(X_train, y_train, best_params=None, random_state=42, n_jobs=-1, cv=5):
    """
    Apply Random Forest with resource measurements, including memory, training time, and CPU usage.

    Parameters:
        X_train: Training features.
        y_train: Training labels.
        best_params: Dictionary of best parameters for Random Forest.
        random_state: Random state for reproducibility.
        n_jobs: Number of jobs for parallel processing.
        cv: Number of cross-validation folds.

    Returns:
        cv_scores_rf: Cross-validation scores.
        measurement_rf: Dictionary of memory, training time, and CPU usage.
        rf_model: Trained Random Forest model.
    """

    measurement_rf = {}

    # Default to empty dictionary if best_params is not provided
    best_params = best_params or {}

    rf_model = RandomForestClassifier(**best_params, random_state=random_state, n_jobs=n_jobs)

    # Function to monitor CPU usage during training
    cpu_usage = []
    stop_flag = threading.Event()

    def monitor_cpu():
        while not stop_flag.is_set():
            cpu_usage.append(psutil.cpu_percent(interval=0.1))

    # Function to train the model
    def train_model():
        rf_model.fit(X_train, y_train)

    try:
        # Start CPU monitoring in a separate thread
        cpu_thread = threading.Thread(target=monitor_cpu)
        cpu_thread.start()

        # Measure memory usage and training time
        start_time = time.time()
        train_memory_rf = max(memory_usage((train_model,)))  # Measure peak memory usage
        training_time = time.time() - start_time

        # Stop CPU monitoring
        stop_flag.set()
        cpu_thread.join()

        # Add measurements
        measurement_rf['Memory Usage (MB)'] = train_memory_rf
        measurement_rf['Training Time (s)'] = training_time
        measurement_rf['Peak CPU Usage (%)'] = max(cpu_usage)
        measurement_rf['Average CPU Usage (%)'] = sum(cpu_usage) / len(cpu_usage) if cpu_usage else 0

        # Perform cross-validation
        cv_scores_rf = cross_val_score(rf_model, X_train, y_train, cv=cv, n_jobs=n_jobs)

        return cv_scores_rf, measurement_rf, rf_model

    except Exception as e:
        print(f"Error during Random Forest training: {e}")
        return None, None, None


In [ ]:
# Apply XGBoost with resource measurements
def apply_xgboost(X_train, y_train, best_params=None, random_state=42, n_jobs=-1, cv=5):
    """
    Apply XGBoost with resource measurements, including memory, training time, and CPU usage.

    Parameters:
        X_train: Training features.
        y_train: Training labels.
        best_params: Dictionary of best parameters for XGBoost.
        random_state: Random state for reproducibility.
        cv: Number of cross-validation folds.

    Returns:
        cv_scores_xgb: Cross-validation scores.
        measurement_xgb: Dictionary of memory, training time, and CPU usage.
        xgb_model: Trained XGBoost model.
    """

    measurement_xgb = {}

    # Default to empty dictionary if best_params is not provided
    best_params = best_params or {}

    xgb_model = xgb.XGBClassifier(**best_params, objective='multi:softmax', num_class=len(y_train.unique()), random_state=random_state, n_jobs=n_jobs)

    # Function to monitor CPU usage during training
    cpu_usage = []
    stop_flag = threading.Event()

    def monitor_cpu():
        while not stop_flag.is_set():
            cpu_usage.append(psutil.cpu_percent(interval=0.1))

    # Function to train the model
    def train_model():
        xgb_model.fit(X_train, y_train)

    try:
        # Start CPU monitoring in a separate thread
        cpu_thread = threading.Thread(target=monitor_cpu)
        cpu_thread.start()

        # Measure memory usage and training time
        start_time = time.time()
        train_memory_xgb = max(memory_usage((train_model,)))  # Measure peak memory usage
        training_time = time.time() - start_time

        # Stop CPU monitoring
        stop_flag.set()
        cpu_thread.join()

        # Add measurements
        measurement_xgb['Memory Usage (MB)'] = train_memory_xgb
        measurement_xgb['Training Time (s)'] = training_time
        measurement_xgb['Peak CPU Usage (%)'] = max(cpu_usage)
        measurement_xgb['Average CPU Usage (%)'] = sum(cpu_usage) / len(cpu_usage) if cpu_usage else 0

        # Perform cross-validation
        cv_scores_xgb = cross_val_score(xgb_model, X_train, y_train, cv=cv, n_jobs=n_jobs)

        return cv_scores_xgb, measurement_xgb, xgb_model

    except Exception as e:
        print(f"Error during XGBoost training: {e}")
        return None, None, None


In [ ]:
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import cross_val_score

# Add this NEW helper function before your existing apply_rf, apply_xgboost, apply_knn functions

def cross_val_with_resampling(model, X, y, resampler=None, cv=5, n_jobs=-1):
    """
    Perform cross-validation with resampling (SMOTE/undersampling) inside each fold.
    This prevents data leakage from synthetic samples.

    Parameters:
        model: The classifier to evaluate
        X: Training features
        y: Training labels
        resampler: Resampling strategy (SMOTE, RandomUnderSampler, etc.). If None, no resampling.
        cv: Number of cross-validation folds
        n_jobs: Number of parallel jobs

    Returns:
        cv_scores: Array of cross-validation scores
    """
    if resampler is not None:
        # Create pipeline that applies resampling INSIDE each CV fold
        pipeline = ImbPipeline([
            ('resampler', resampler),
            ('classifier', model)
        ])
    else:
        pipeline = model

    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X, y, cv=cv, n_jobs=n_jobs, scoring='accuracy')

    return cv_scores


# MODIFIED apply_rf function
def apply_rf(X_train, y_train, best_params=None, random_state=42, n_jobs=-1, cv=5,
             use_resampling=False, resampler=None):
    """
    Apply Random Forest with resource measurements, including memory, training time, and CPU usage.

    NEW PARAMETER:
        use_resampling: If True, apply resampling inside CV folds (prevents leakage)
        resampler: The resampling strategy to use (e.g., SMOTE, RandomUnderSampler)
    """

    measurement_rf = {}
    best_params = best_params or {}

    rf_model = RandomForestClassifier(**best_params, random_state=random_state, n_jobs=n_jobs)

    # Function to monitor CPU usage during training
    cpu_usage = []
    stop_flag = threading.Event()

    def monitor_cpu():
        while not stop_flag.is_set():
            cpu_usage.append(psutil.cpu_percent(interval=0.1))

    # Function to train the model
    def train_model():
        rf_model.fit(X_train, y_train)

    try:
        # Start CPU monitoring in a separate thread
        cpu_thread = threading.Thread(target=monitor_cpu)
        cpu_thread.start()

        # Measure memory usage and training time
        start_time = time.time()
        train_memory_rf = max(memory_usage((train_model,)))
        training_time = time.time() - start_time

        # Stop CPU monitoring
        stop_flag.set()
        cpu_thread.join()

        # Add measurements
        measurement_rf['Memory Usage (MB)'] = train_memory_rf
        measurement_rf['Training Time (s)'] = training_time
        measurement_rf['Peak CPU Usage (%)'] = max(cpu_usage) if cpu_usage else 0
        measurement_rf['Average CPU Usage (%)'] = sum(cpu_usage) / len(cpu_usage) if cpu_usage else 0

        # Perform cross-validation with proper resampling
        if use_resampling and resampler is not None:
            # Use the new function that applies resampling inside CV
            cv_scores_rf = cross_val_with_resampling(
                RandomForestClassifier(**best_params, random_state=random_state, n_jobs=n_jobs),
                X_train,
                y_train,
                resampler=resampler,
                cv=cv,
                n_jobs=n_jobs
            )
        else:
            # Standard CV without resampling
            cv_scores_rf = cross_val_score(rf_model, X_train, y_train, cv=cv, n_jobs=n_jobs)

        return cv_scores_rf, measurement_rf, rf_model

    except Exception as e:
        print(f"Error during Random Forest training: {e}")
        return None, None, None

In [ ]:
# Apply KNN with resource measurements
def apply_knn(X_train, y_train, best_params=None, n_jobs=-1, cv=5):
    """
    Apply K-Nearest Neighbors (KNN) with resource measurements, including memory, training time, and CPU usage.

    Parameters:
        X_train: Training features.
        y_train: Training labels.
        best_params: Dictionary of best parameters for KNN.
        cv: Number of cross-validation folds.

    Returns:
        cv_scores_knn: Cross-validation scores.
        measurement_knn: Dictionary of memory, training time, and CPU usage.
        knn_model: Trained KNN model.
    """

    measurement_knn = {}

    # Default to empty dictionary if best_params is not provided
    best_params = best_params or {}

    # Initialize KNN model
    knn_model = KNeighborsClassifier(**best_params, n_jobs=n_jobs)

    # Function to monitor CPU usage during training
    cpu_usage = []
    stop_flag = threading.Event()

    def monitor_cpu():
        while not stop_flag.is_set():
            cpu_usage.append(psutil.cpu_percent(interval=0.1))

    # Function to train the model
    def train_model():
        knn_model.fit(X_train, y_train)

    try:
        # Start CPU monitoring in a separate thread
        cpu_thread = threading.Thread(target=monitor_cpu)
        cpu_thread.start()

        # Measure memory usage and training time
        start_time = time.time()
        train_memory_knn = max(memory_usage((train_model,)))  # Measure peak memory usage
        training_time = time.time() - start_time

        # Stop CPU monitoring
        stop_flag.set()
        cpu_thread.join()

        # Add measurements
        measurement_knn['Memory Usage (MB)'] = train_memory_knn
        measurement_knn['Training Time (s)'] = training_time
        measurement_knn['Peak CPU Usage (%)'] = max(cpu_usage)
        measurement_knn['Average CPU Usage (%)'] = sum(cpu_usage) / len(cpu_usage) if cpu_usage else 0

        # Perform cross-validation
        cv_scores_knn = cross_val_score(knn_model, X_train, y_train, cv=cv, n_jobs=n_jobs)

        return cv_scores_knn, measurement_knn, knn_model

    except Exception as e:
        print(f"Error during KNN training: {e}")
        return None, None, None


# 1. Loading and Preparing

In [ ]:
# Loading the dataset
clean_df = pd.read_csv('/content/drive/MyDrive/Data/cicids2017_cleaned.csv')

In [ ]:
# ===== TEMPORAL SPLIT FIX =====
# Check if your dataset has a timestamp or date column
# If you have a timestamp column (e.g., 'Timestamp'), use this:

# Option 1: If you have a timestamp column
if 'Timestamp' in clean_df.columns:
    # Sort by timestamp to ensure chronological order
    clean_df = clean_df.sort_values('Timestamp').reset_index(drop=True)

    # Calculate split index (70% train, 30% test)
    split_idx = int(len(clean_df) * 0.7)

    # Split chronologically
    train_df = clean_df.iloc[:split_idx]
    test_df = clean_df.iloc[split_idx:]

    # Separate features and target
    X_train = train_df.drop(['Attack Type', 'Timestamp'], axis=1)  # Drop timestamp from features
    y_train = train_df['Attack Type']
    X_test = test_df.drop(['Attack Type', 'Timestamp'], axis=1)
    y_test = test_df['Attack Type']

    print(f"Train set: {len(train_df)} samples")
    print(f"Test set: {len(test_df)} samples")
    print(f"\nTrain date range: {train_df['Timestamp'].min()} to {train_df['Timestamp'].max()}")
    print(f"Test date range: {test_df['Timestamp'].min()} to {test_df['Timestamp'].max()}")

# Option 2: If you DON'T have a timestamp (but data is already chronologically ordered)
else:
    print("WARNING: No timestamp column found. Assuming data is already in chronological order.")
    print("If data was shuffled during preprocessing, this will cause data leakage!")

    # Preparing features and target
    X = clean_df.drop('Attack Type', axis=1)
    y = clean_df['Attack Type']

    # Calculate split index (70% train, 30% test)
    split_idx = int(len(clean_df) * 0.7)

    # Split chronologically (assumes data is already sorted by time)
    X_train = X.iloc[:split_idx]
    y_train = y.iloc[:split_idx]
    X_test = X.iloc[split_idx:]
    y_test = y.iloc[split_idx:]

    print(f"Train set: {len(X_train)} samples")
    print(f"Test set: {len(X_test)} samples")

# Check class distribution after temporal split
print("\nTraining set class distribution:")
print(y_train.value_counts())
print("\nTest set class distribution:")
print(y_test.value_counts())

In [ ]:
clean_df.head()

## 1.1. Preparing Training and Test Sets

In [ ]:
# Preparing training and test splits
X = clean_df.drop('Attack Type', axis=1)
y = clean_df['Attack Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

### 1.1.1. Feature Scaling

Feature scaling is essential for ensuring that all features contribute equally to the model's performance, improving convergence speed, and enhancing the accuracy of machine learning algorithms, particularly those that rely on distance metrics. The first step is to decide on the appropriate type of scaling. To do this, examining outliers and the distribution of features is an effective approach.

* StandardScaler is best for normally distributed data.
* MinMaxScaler is best for bounded features.
* Robust Scaler is best when outliers are present, ensuring that the scaling maintains the robustness of the data.


| **Aspect**             | **StandardScaler**                          | **MinMaxScaler**                          | **RobustScaler**                          |  
|-----------------------|--------------------------------------------|------------------------------------------|-------------------------------------------|  
| **Scaling Approach**  | Mean and standard deviation                 | Minimum and maximum                      | Median and interquartile range            |  
| **Sensitivity to Outliers** | Sensitive (affected by outliers)       | Sensitive (max and min can distort)     | Not sensitive (robust to outliers)        |  
| **Range After Scaling** | Normal distribution (mean=0, sd=1)      | Custom range (default [0,1])            | No fixed range (median is used)           |  
| **Best For**          | Normal distributions                        | Bounded data                             | Data with outliers or non-Gaussian types  |

Based on the [exploratory data analysis (EDA)](https://www.kaggle.com/code/ericanacletoribeiro/cicids2017-comprehensive-data-processing-for-ml), most features exhibit a high percentage of outliers, with values exceeding 10%. As a result, Robust Scaling is likely the most appropriate preprocessing technique. Additionally, statistical tests, such as the Anderson-Darling test (also available on the beforementioned Notebook), indicated that the null hypothesis was rejected for all of the features, confirming that the data does not follow a normal distribution.

In [ ]:
# Initialize RobustScaler
scaler = RobustScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test set using the fitted scaler
X_test_scaled = scaler.transform(X_test)

| **Model**                  | **Is Scaling Important?** | **Explanation**                                                                 |
|----------------------------|---------------------------|---------------------------------------------------------------------------------|
| **Random Forest**          | No                       | Random Forest is tree-based and works with the raw values without being affected by scaling. |
| **XGBoost**                | No                       | Like Random Forest, XGBoost is tree-based and insensitive to feature scaling.   |
| **KNN**    | Yes                      | KNN uses distance metrics, which are affected by the scale of features. |

In [ ]:
# Exporting the scaler in case it is needed during deployment
joblib.dump(scaler, 'robust_scaler.joblib')

### 1.1.2. Resampling Techniques

In [ ]:
# Checking the distribution of the target variable
clean_df['Attack Type'].value_counts()

The dataset shows a considerable imbalance, as already discussed on the [previous notebook](https://www.kaggle.com/datasets/ericanacletoribeiro/cicids2017-cleaned-and-preprocessed). Since cross-validation results are already available without addressing this issue (serving as a benchmark), the decision here is to undersample the majority class to reduce complexity while aiming to maintain performance. This can help the model pay more attention to the less frequent attack types, which are often the focus in intrusion detection systems.

In [ ]:
# Initializing the undersampling for the clean df
X_train_resampled, y_train_resampled = RandomUnderSampler(sampling_strategy={'Normal Traffic': 500000}, random_state=42).fit_resample(X_train, y_train)

# Initializing the undersampling for the scaled df
X_train_scaled, y_train_scaled = RandomUnderSampler(sampling_strategy={'Normal Traffic': 500000}, random_state=42).fit_resample(X_train_scaled, y_train)

In order to further balance the training set, we can also partially oversample the minority classes with SMOTE. _Synthetic Minority Over-sampling Technique_ is a widely-used method to address class imbalance in datasets by generating synthetic samples for the minority class, rather than merely duplicating existing ones. The values were chosen based on the value_counts above: all in all, rouding up the numbers in order to avoid an overly aggressive oversampling.

In [ ]:
# Initializing the oversampling for the scaled df
X_train_resampled_scaled, y_train_resampled_scaled = SMOTE(sampling_strategy={'Bots': 2000, 'Web Attacks': 2000, 'Brute Force': 7000, 'Port Scanning': 70000, 'DDoS':90000, 'DoS': 200000}, random_state=42).fit_resample(X_train_scaled, y_train_scaled)

Since SMOTE relies on distance calculations to generate synthetic samples, it is essential to scale the dataset beforehand to ensure that all features contribute equally to the interpolation process and avoid skewed or meaningless synthetic data. Therefore, this process will not be applied to the raw training data.

In [ ]:
# Cleaning up
del X_train_scaled, X_train, y_train, X, y, clean_df

In [ ]:
# Checking the distribution of the attack types in the resampled/raw training set
y_train_scaled.value_counts()

In [ ]:
# Checking the distribution of the attack types in the resampled/scaled training set
y_train_resampled_scaled.value_counts()

### 1.1.3. A Note on PCA

During the experimentation process, PCA (Principal Component Analysis) was considered as a dimensionality reduction technique to speed up model training and improve computational efficiency. While PCA proved effective in accelerating training for algorithms like KNN, it introduced an additional preprocessing step in the pipeline, which is not ideal for real-time production environments. Production systems prioritize simplicity and speed, and the need to transform new data using PCA before making predictions could potentially impact real-time performance.

Moreover, the dataset exhibited strong correlations among features, causing PCA to converge the variance into just one or two principal components. While this initially seemed promising due to the high explained variance ratio, these components failed to fully capture the intricacies of the data. This led to poorer results in terms of model performance, as the reduced representation lacked the necessary granularity to differentiate between classes effectively. Based on these observations, PCA was ultimately excluded from the workflow to maintain a balance between computational efficiency, simplicity, and model performance on production.

# 2. Supervised Learning

## 2.1. Random Forest

Random Forest is robust to variance in the data. Therefore, it can be trained on the non-scaled version of the dataset without a significant impact on performance.

### 2.1.1. Hyperparameter Tuning*

To determine the optimal parameters for machine learning models, it's crucial to evaluate how different hyperparameters affect performance. A good way to do this is by visualizing the model's performance across various configurations, using techniques such as RandomizedSearchCV or GridSearchCV.

While an exhaustive grid search is often recommended for thorough hyperparameter optimization through cross-validation (CV), this process can be computationally expensive and time-consuming. Given the performance constraints of this project, the goal here is to simplify the search by using RandomizedSearchCV. This approach randomly samples parameter combinations and checks if they can outperform the standard parameter settings of each model.

***Due to resources limitations on Kaggle, this section was run locally, results are shown at the end**

In [ ]:
 # Defining the parameters for the Random Forest Classifier
 #param_grid = {
#     'n_estimators': [100, 150, 200],
#     'max_depth': [20, 30, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['sqrt', 'log2'],
# }

In [ ]:
# Creating the Random Forest Classifier
#rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
# # Saving results with the standard parameters
#cv_sc_rf = cross_val_score(rf_model, X_train_resampled, y_train_resampled, cv=3, n_jobs=-1)
#cv_sc_rf = np.mean(cv_sc_rf)

 # Apply RandomizedSearchCV
#random_search_rf = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, n_iter=20, cv=3, n_jobs=-1, verbose=2)
#random_search_rf.fit(X_train_resampled, y_train_resampled)

In [ ]:
 # Get the best parameters
 #print(f'Best Parameters: {random_search_rf.best_params_}')
 #print(f"Best Cross-Validation Score: {random_search_rf.best_score_}")
 #print(f"Cross-Validation from Standard: {cv_sc_rf}")

 #best_params_rf = random_search_rf.best_params_ if random_search_rf.best_score_ > cv_sc_rf else None

# del random_search_rf

**A Note on Hyperparameter Tuning**

Due to the stochastic nature of RandomizedSearchCV, the values for each parameter might differ every time this notebook is run. To ensure reproducibility, the following parameters were selected as the best-performing ones for the Random Forest model in this experiment:

Best Parameters:
- `n_estimators`: 200
- `min_samples_split`: 5
- `min_samples_leaf`: 2
- `max_features`: 'sqrt'
- `max_depth`: None

If you wish to replicate my results, these are the parameters that provided the optimal performance for this model.

In [ ]:
# Defining manually due to Kaggle performance limitations
best_params_rf = {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None}

### 2.1.2. Fitting the Random Forest

In [ ]:
# Fitting the model
cv_scores_rf, measurement_rf, rf_model = apply_rf(X_train_resampled, y_train_resampled, best_params=best_params_rf)

# Making predictions
y_pred_rf = rf_model.predict(X_test)

In [ ]:
# Evaluating the model performance on the cross validation set vs accuracy on the test set
cv_scores_mean_rf = np.mean(cv_scores_rf)
print(f'Cross validation average score: {cv_scores_mean_rf:.4f} +/- standard deviation: {np.std(cv_scores_rf):.4f}')

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Accuracy on the test set: {accuracy_rf:.4f}')

In [ ]:
# Checking computational cost
print("Resource measurements:", measurement_rf)

In [ ]:
# Evaluating the model via confusion matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)

plt.figure(figsize=(10, 7))
sns.heatmap(cm_rf, annot=True, fmt='d', xticklabels=rf_model.classes_, yticklabels=rf_model.classes_, cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.title('Random ForestConfusion Matrix')
plt.show()

In [ ]:
# Classification report
print(classification_report(y_test, y_pred_rf))

With a mean cross-validation score of 0.9987 and an accuracy score of 0.9988, the model demonstrates strong performance without signs of underfitting or overfitting (your numbers might differ due to different results from RamdomizedSearchCV). Additionally, the confusion matrix reveals a significant improvement in recall (from 0.74 to 0.83) for the 'Bots' class compared to the benchmark model ([available here](https://www.kaggle.com/code/ericanacletoribeiro/cicids2017-comprehensive-data-processing-for-ml)).
While the precision score for the same class decreased from 0.86 to 0.71, the trade-off is justified in the context of a NIDS. In such systems, minimizing false negatives is crucial, as undetected threats can compromise network security. False positives, while inconvenient, are less harmful than false negatives. This improvement in recall indicates that the model is now better at detecting 'Bots' attacks, validating the effectiveness of fine-tuning techniques for this dataset.

### 2.1.3. Exporting the Model

In [ ]:
# Save the model to a file
joblib.dump(rf_model, 'random_forest.joblib')

## 2.2. XGBoost

XGBoost (Extreme Gradient Boosting) is a highly efficient implementation of gradient boosting, which is known for its power in handling imbalanced datasets, preventing overfitting, and its ability to work well with both structured and unstructured data. In this section, we will evaluate XGBoost for the task of anomaly-based intrusion detection.

While Random Forest is an ensemble method that builds multiple decision trees independently, XGBoost takes a more iterative approach, building trees sequentially, where each new tree attempts to correct errors made by the previous one. This results in a model that often outperforms Random Forest in terms of predictive accuracy, particularly for complex datasets.

### 2.2.1. Hyperparameter Tuning*

***Due to resources limitations on Kaggle, this section was run locally, results are showns at the end**

In [ ]:
# # Defining the parameter grid for XGBoost
# param_dist = {
#     'n_estimators': [100, 150, 200],
#     'max_depth': [3, 6, 9],
#     'learning_rate': [0.2, 0.3, 0.4],
#     'subsample': [0.7, 0.8, 1.0],
#     'colsample_bytree': [0.7, 0.8, 1.0],
#     'min_child_weight': [1, 5, 10],
# }

XGBoost's `multi:softmax` objective requires numerical labels for classification. Therefore, a mapping is necessary to convert categorical labels into numerical values before training the model.

In [ ]:
# # Creating the XGBoost Classifier
# xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(y_train_resampled.unique()), random_state=42, n_jobs=-1)

# Custom mapping for the attack types
label_mapping = {
    'Normal Traffic': 0,
    'DoS': 1,
    'DDoS': 2,
    'Port Scanning': 3,
    'Brute Force': 4,
    'Web Attacks': 5,
    'Bots': 6
}
y_train_resampled_mapped = y_train_resampled.map(label_mapping)
y_test_mapped = y_test.map(label_mapping)

In [ ]:
 # Saving results with the standard parameters
 #cv_sc_xgb = cross_val_score(xgb_model, X_train_resampled, y_train_resampled_mapped, cv=3, n_jobs=-1)
 #cv_sc_xgb = np.mean(cv_sc_xgb)

 # Perform RandomizedSearchCV
# random_search_xgb = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=30, cv=3, n_jobs=-1, verbose=2, random_state=42)
# random_search_xgb.fit(X_train_resampled, y_train_resampled_mapped)

In [ ]:
 # Best parameters found by RandomizedSearchCV
 #print(f'Best Parameters for XGBoost: {random_search_xgb.best_params_}')
 #print(f"Best Cross-Validation Score: {random_search_xgb.best_score_}")
# print(f"Cross-Validation from Standard: {cv_sc_xgb}")

# best_params_xgb = random_search_xgb.best_params_ if random_search_xgb.best_score_ > cv_sc_xgb else None

# del random_search_xgb

**A Note on Hyperparameter Tuning**

Due to the stochastic nature of RandomizedSearchCV, the values for each parameter might differ every time this notebook is run. To ensure reproducibility, the following parameters were selected as the best-performing ones for the XGBoost model in this experiment:

Best Parameters:
- `subsample`: 1.0
- `n_estimators`: 150
- `min_child_weight`: 1
- `max_depth`: 3
- `learning_rate`: 0.3
- `colsample_bytree`: 0.7

If you wish to replicate my results, these are the parameters that provided the optimal performance for this model.


In [ ]:
# Defining parameters manually due to Kaggles performance limitataions
best_params_xgb = {'subsample': 1.0, 'n_estimators': 150, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.3, 'colsample_bytree': 0.7}

### 2.2.2. Fitting the XGBoost

In [ ]:
# Fitting the model
cv_scores_xgb, measurement_xgb, xgb_model = apply_xgboost(X_train_resampled, y_train_resampled_mapped, best_params=best_params_xgb)

# Making predictions
y_pred_xgb = xgb_model.predict(X_test)

In [ ]:
# Evaluating the model performance on the cross validation set vs accuracy on the test set
cv_scores_mean_xgb = np.mean(cv_scores_xgb)
print(f'Cross validation average score: {cv_scores_mean_xgb:.4f} +/- standard deviation: {np.std(cv_scores_xgb):.4f}')

accuracy_xgb = accuracy_score(y_test_mapped, y_pred_xgb)
print(f'Accuracy on the test set: {accuracy_xgb:.4f}')

In [ ]:
# Checking computational cost
print("Resource measurements:", measurement_xgb)

In [ ]:
# Remapping the labels for visualization
reverse_label_mapping = {v: k for k, v in label_mapping.items()}
predicted_labels = [reverse_label_mapping[pred] for pred in y_pred_xgb]
actual_labels = sorted([reverse_label_mapping[label] for label in xgb_model.classes_])

# Confusion matrix
cm_xgb = confusion_matrix(y_test, predicted_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(cm_xgb, annot=True, fmt='d', xticklabels=actual_labels, yticklabels=actual_labels, cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.title('XGBoost Confusion Matrix')
plt.show()

In [ ]:
# Classification report
print(classification_report(y_test, predicted_labels))

The XGBoost model demonstrated a significant improvement in recall for the 'Bots' class compared to the Random Forest model. Recall is a critical metric for this application, as undetected attacks can compromise network security. The difference between the two models across other metrics, including accuracy, precision, and overall F1-score, remains negligible, with a marginal advantage to XGB.

### 2.2.3. Exporting the Model

In [ ]:
# Save the model to a file
joblib.dump(xgb_model, 'xgboost.joblib')

## 2.3. K-Nearest Neighbors (KNN)

K-Nearest Neighbors (KNN) is a simple and effective classification algorithm that works by finding the majority class among the `k` nearest data points for a given instance. It is sensitive to the scale of the data, so it is important to ensure that the features are standardized (scaled).

### 2.3.1. Hyperparamenter Tuning*

***Due to resources limitations on Kaggle, this section was run locally, results are showns at the end**

In [ ]:
# # Defining the parameters for KNN
#param_grid_knn = {
#     'n_neighbors': [3, 5, 7],
#     'weights': ['uniform', 'distance'],
 #}

In [ ]:
 # Creating the KNN model
 #knn_model = KNeighborsClassifier(n_jobs=-1)

In [ ]:
 # Saving results with the standard parameters
 #cv_sc_knn = cross_val_score(knn_model, X_train_resampled_scaled, y_train_resampled_scaled, cv=3, n_jobs=-1)
 #cv_sc_knn = np.mean(cv_sc_knn)

 # Apply RandomizedSearchCV
 #random_search_knn = RandomizedSearchCV(estimator=knn_model, param_distributions=param_grid_knn, n_iter=6, cv=3, n_jobs=-1, verbose=2)
 #random_search_knn.fit(X_train_resampled_scaled, y_train_resampled_scaled)

In [ ]:
 # Get the best parameters
# print(f'Best Parameters: {random_search_knn.best_params_}')
# print(f"Best Cross-Validation Score: {random_search_knn.best_score_}")
 #print(f"Cross-Validation from Standard: {cv_sc_knn}")

 #best_params_knn = random_search_knn.best_params_ if random_search_knn.best_score_ > cv_sc_knn else None

# del random_search_knn

Best Parameters:

- `weights`: distance
- `n_neighbours`: 3

In [ ]:
# Defining manually
best_params_knn = {'weights': 'distance', 'n_neighbors': 3}

### 2.3.2. Fitting the KNN

In [ ]:
# Fitting the model
cv_scores_knn, measurement_knn, knn_model = apply_knn(X_train_resampled_scaled, y_train_resampled_scaled, best_params=best_params_knn)

# Making predictions
y_pred_knn = knn_model.predict(X_test_scaled)

In [ ]:
# Evaluating the model performance on the cross-validation set vs accuracy on the test set
cv_scores_mean_knn = np.mean(cv_scores_knn)
print(f'Cross-validation average score: {cv_scores_mean_knn:.4f} +/- standard deviation: {np.std(cv_scores_knn):.4f}')

accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f'Accuracy on the test set: {accuracy_knn:.4f}')

In [ ]:
# Checking computational cost
print("Resource measurements:", measurement_knn)

In [ ]:
# Evaluating the model via confusion matrix
cm_knn = confusion_matrix(y_test, y_pred_knn)

plt.figure(figsize=(10, 7))
sns.heatmap(cm_knn, annot=True, fmt='d', xticklabels=knn_model.classes_, yticklabels=knn_model.classes_, cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.title('KNN Confusion Matrix')
plt.show()

In [ ]:
# Classification report
print(classification_report(y_test, y_pred_knn))

The KNN model also demonstrated strong performance, though it was slightly outperformed by the previous models. However, it is worth noting that the 'Bots' class remains the weakest in the dataset, with consistently lower values across all evaluation metrics.

### 2.1.3. Exporting the Model

In [ ]:
# Save the model to a file
joblib.dump(knn_model, 'knn_model.joblib')

# 3. Comparing Performance Results

In [ ]:
from sklearn.metrics import roc_auc_score

# Calculate ROC-AUC scores for each model
y_pred_proba_rf = rf_model.predict_proba(X_test)
roc_auc_rf = roc_auc_score(y_test, y_pred_proba_rf, multi_class='ovr', average='weighted')

y_pred_proba_xgb = xgb_model.predict_proba(X_test)
roc_auc_xgb = roc_auc_score(y_test_mapped, y_pred_proba_xgb, multi_class='ovr', average='weighted')

y_pred_proba_knn = knn_model.predict_proba(X_test_scaled)
roc_auc_knn = roc_auc_score(y_test, y_pred_proba_knn, multi_class='ovr', average='weighted')

print(f'ROC-AUC Random Forest: {roc_auc_rf:.4f}')
print(f'ROC-AUC XGBoost: {roc_auc_xgb:.4f}')
print(f'ROC-AUC KNN: {roc_auc_knn:.4f}')


In [ ]:
# Calculating precision, recall, and F1 score for each model
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

precision_xgb = precision_score(y_test_mapped, y_pred_xgb, average='weighted')
recall_xgb = recall_score(y_test_mapped, y_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test_mapped, y_pred_xgb, average='weighted')

precision_knn = precision_score(y_test, y_pred_knn, average='weighted')
recall_knn = recall_score(y_test, y_pred_knn, average='weighted')
f1_knn = f1_score(y_test, y_pred_knn, average='weighted')

In [ ]:
# Creating the results dataframe
supervised_results = pd.DataFrame({
    'Model': ['Random Forest', 'XGBoost', 'KNN'],
    'Accuracy': [accuracy_rf, accuracy_xgb, accuracy_knn],
    'Cross Validation Mean': [cv_scores_mean_rf, cv_scores_mean_xgb, cv_scores_mean_knn],
    'Precision': [precision_rf, precision_xgb, precision_knn],
    'Recall': [recall_rf, recall_xgb, recall_knn],
    'F1 Score': [f1_rf, f1_xgb, f1_knn],
    'ROC-AUC': [roc_auc_rf, roc_auc_xgb, roc_auc_knn],  # ADD THIS LINE
    'Memory Usage (MB)': [measurement_rf['Memory Usage (MB)'], measurement_xgb['Memory Usage (MB)'], measurement_knn['Memory Usage (MB)']],
    'Training Time (s)': [measurement_rf['Training Time (s)'], measurement_xgb['Training Time (s)'], measurement_knn['Training Time (s)']],
    'Peak CPU Usage (%)': [measurement_rf['Peak CPU Usage (%)'], measurement_xgb['Peak CPU Usage (%)'], measurement_knn['Peak CPU Usage (%)']],
    'Average CPU Usage (%)': [measurement_rf['Average CPU Usage (%)'], measurement_xgb['Average CPU Usage (%)'], measurement_knn['Average CPU Usage (%)']],
})

In [ ]:
# Plotting the comparison for accuracy, cross-validation, and metrics
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plotting Accuracy and Cross Validation Mean
supervised_results.set_index('Model')[['Accuracy', 'Cross Validation Mean']].plot(kind='bar', ax=axes[0, 0], color=['skyblue', 'lightgreen'], legend=True)
axes[0, 0].set_title('Model Comparison: Accuracy and Cross Validation Mean')
axes[0, 0].set_ylabel('Score')
axes[0, 0].set_xlabel('Model')
axes[0, 0].set_ylim(0.95, 1.0)
axes[0, 0].legend(loc='lower left')

# Plotting Precision, Recall, F1 Score, and ROC-AUC (MODIFIED)
supervised_results.set_index('Model')[['Precision', 'Recall', 'F1 Score', 'ROC-AUC']].plot(kind='bar', ax=axes[0, 1], color=['orange', 'lightcoral', 'yellowgreen', 'mediumpurple'], legend=True)
axes[0, 1].set_title('Model Comparison: Precision, Recall, F1 Score, ROC-AUC')
axes[0, 1].set_ylabel('Score')
axes[0, 1].set_xlabel('Model')
axes[0, 1].set_ylim(0.95, 1.0)
axes[0, 1].legend(loc='lower left')

# Plotting Memory Usage and Training Time
ax1 = axes[1, 0]

supervised_results.set_index('Model')['Memory Usage (MB)'].plot(
    kind='bar', ax=ax1, color='lightblue', label='Memory Usage (MB)', width=0.6
)
ax1.set_ylabel('Memory Usage (MB)', color='lightblue')
ax1.tick_params(axis='y', labelcolor='lightblue')

ax2 = ax1.twinx()
supervised_results.set_index('Model')['Training Time (s)'].plot(
    ax=ax2, color='lightpink', marker='o', label='Training Time (s)'
)
ax2.set_ylabel('Training Time (s)', color='lightpink')
ax2.tick_params(axis='y', labelcolor='lightpink')

ax1.set_title('Model Comparison: Memory Usage and Training Time')
ax1.set_xlabel('Model')

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper right')

# Plotting Peak and Average CPU Usage
supervised_results.set_index('Model')[['Peak CPU Usage (%)', 'Average CPU Usage (%)']].plot(kind='bar', ax=axes[1, 1], color=['lightgreen', 'salmon'], legend=True)
axes[1, 1].set_title('Model Comparison: CPU Usage')
axes[1, 1].set_ylabel('Percentage')
axes[1, 1].set_xlabel('Model')
axes[1, 1].legend(loc='lower left')

plt.tight_layout()
plt.show()

[link text](https://)The plots effectively illustrate the performance of each model during training and testing phases. While Random Forest (RF) and XGBoost (XGB) achieved the highest accuracy and test scores, they also demanded more computational power. On the other hand, it's important to remember that KNN is a 'lazy learner,' meaning that actual training occurs when new data is introduced, either during testing or in production. The true performance of KNN will become clearer once the prototype is deployed and tested in real-time.

In [ ]:
# Plot comparative analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Performance metrics
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
performance_data = supervised_results[['Model'] + metrics].set_index('Model')
performance_data.plot(kind='bar', ax=ax1)
ax1.set_title('Performance Metrics Comparison')
ax1.set_ylim(0.9, 1.0)

# Resource usage
resources = ['Memory Usage (MB)', 'Training Time (s)']
resource_data = supervised_results[['Model'] + resources].set_index('Model')
resource_data.plot(kind='bar', ax=ax2)
ax2.set_title('Resource Usage Comparison')

# CPU usage
cpu_metrics = ['Peak CPU Usage (%)', 'Average CPU Usage (%)']
cpu_data = supervised_results[['Model'] + cpu_metrics].set_index('Model')
cpu_data.plot(kind='bar', ax=ax3)
ax3.set_title('CPU Usage Comparison')

# Cross-validation comparison
cv_data = supervised_results[['Model', 'Cross Validation Mean']].set_index('Model')
cv_data.plot(kind='bar', ax=ax4, legend=False)
ax4.set_title('Cross-Validation Performance')
ax4.set_ylim(0.98, 1.0)

plt.tight_layout()
plt.show()